# Gaia Spatial Stats Examples

This document contains examples of Gaia Spatial Stats Plugin's geospatial processes.
The original Python notebook is available in the Gaia Spatial Stats Plugin code under the docs/examples folder.

In [2]:
%matplotlib inline
# Import required modules
from gaia.geo.geo_inputs import VectorFileIO
from gaia import formats
from gaia_spatialstats.processes import ClusterProcess, WeightProcess, \
    AutocorrelationProcess, GearyCProcess, GammaProcess, ClassifierProcess

# for mapping
import folium
from folium.plugins import ImageOverlay

In [3]:
# Create VectorFileIO object from files containing GeoJSON
fp = '../../tests/data/baghdad_hospitals.geojson'
baghdad_hospitals = VectorFileIO(name='input',
                                 uri=fp)
baghdad_hospitals.read()
print(len(baghdad_hospitals.data))
baghdad_hospitals.data[:5]
# folium.Map(baghdad_hospitals)

89


,DID,DNAME,NID,NNAME,geometry,num_hospitals
0,7,Mansour,74,Baghdad International Airport,(POLYGON ((44.25416164813191 33.36238209600771...,0
1,4,Karadah,46,Zafriniya,(POLYGON ((44.51697757715971 33.23343163393923...,0
2,1,Adhamiyah,25,Rashdiya,(POLYGON ((44.38008196547639 33.43048095703136...,0
3,2,Thawra,28,Thawra City,"(POLYGON ((44.460606439164 33.35860356319699, ...",0
4,3,7 Nissan,33,"Habibiyah, Dor Al-Umal, Baladiyat",(POLYGON ((44.46060643916434 33.35860356319682...,0


# Weight Process

Calculates spatial weight and returns a spatial weight object.
Weight type available includes: contiguity, knnW, distanceBandW, kernel

See PySAL documentation on [Spatial Weights](http://pysal.readthedocs.io/en/latest/users/tutorials/weights.html#spatial-weights)

In [4]:
baghdad_hospitals_weight = WeightProcess('contiguity', inputs=[baghdad_hospitals])
baghdad_hospitals_weight.compute()
weight = baghdad_hospitals_weight.output.read(format=formats.WEIGHT)

Island id:  [84]


In [5]:
print('Number of polygons in geojson')
print(weight.n)
print('\n')

print('Distribution of contiguity spatial weight. (number of neighbors, number of polygons matching criteria)')
print(weight.histogram)
print('\n')

print('Each polygon and associated neighbors')
print(weight.neighbors)

Number of polygons in geojson
89


Distribution of contiguity spatial weight. (number of neighbors, number of polygons matching criteria)
[(0, 1), (1, 0), (2, 4), (3, 8), (4, 17), (5, 24), (6, 16), (7, 13), (8, 4), (9, 1), (10, 1)]


Each polygon and associated neighbors
{0: set([62, 59, 60, 78, 46]), 1: set([88, 86]), 2: set([8, 9, 18, 6]), 3: set([14, 15, 5, 6, 7]), 4: set([32, 5, 11, 12, 15, 22]), 5: set([11, 10, 3, 4]), 6: set([2, 3, 7, 9, 14, 23, 24]), 7: set([8, 9, 3, 14, 6]), 8: set([9, 2, 7]), 9: set([8, 2, 6, 7]), 10: set([11, 5]), 11: set([12, 10, 4, 5]), 12: set([11, 4, 86]), 13: set([39, 40, 41, 51, 52, 29]), 14: set([3, 6, 7, 15, 22, 23, 24]), 15: set([32, 3, 4, 14, 22, 23]), 16: set([17, 25, 22, 38, 23]), 17: set([16, 25, 28, 38, 39]), 18: set([49, 2, 19, 20]), 19: set([49, 18, 20, 21, 24, 27]), 20: set([18, 19]), 21: set([64, 65, 49, 19, 27, 29]), 22: set([32, 4, 14, 15, 16, 23, 25]), 23: set([6, 14, 15, 16, 22, 24, 25]), 24: set([26, 19, 14, 6, 23]), 25: set([16, 17, 22

# Geary's C Process
Calculates Geary's C statistic for spatial autocorrelation for input data. 

Uses contiguity weight (queen) and binary transformation by default.

Returns Geary's C statistics as json

See PySAL documentation on [Geary's C statistics](http://pysal.readthedocs.io/en/latest/library/esda/geary.html)

In [6]:
baghdad_hospitals_gearyc = GearyCProcess('num_hospitals', inputs=[baghdad_hospitals])
baghdad_hospitals_gearyc.compute()
baghdad_hospitals_gearyc_result = baghdad_hospitals_gearyc.output.read(format=formats.JSON)

Island id:  [84]


In [7]:
baghdad_hospitals_gearyc_result

{'C': 1.3975563495214127,
 'EC': 1.0,
 'EC_sim': 1.0077704628893369,
 'p_norm': 3.222677786851591e-07,
 'p_sim': 0.035000000000000003,
 'p_z_sim': 0.047700626811698066,
 'z_sim': 1.6675693160946135}

# Gamma Process

Calculates Gamma Index for spatial autocorrelation for the input data. Uses contiguity weight (queen) and cross product similarity function by default.

Returns Gamma Index attributes as json

See PySAL documentation on [Gamma Statistics](http://pysal.readthedocs.io/en/latest/library/esda/gamma.html)

In [8]:
baghdad_hospitals_gamma = GammaProcess('num_hospitals', inputs=[baghdad_hospitals])
baghdad_hospitals_gamma.compute()
baghdad_hospitals_gamma_result = baghdad_hospitals_gamma.output.read(format=formats.JSON)

Island id:  [84]


In [9]:
baghdad_hospitals_gamma_result

{'g': 28.0,
 'max_g': 88.0,
 'mean_g': 17.713713713713712,
 'min_g': 0.0,
 'p_sim_g': 0.20399999999999999}

# Classifier Process

Classify for choropleth mapping. Default map classifier: Natural_breaks. 

Other options: Equal_Interval, Fisher_Jenks, Fisher_Jenks_Sampled, HeadTail_Breaks, Jenks_Caspall, Jenks_Caspall_Forced, Jenks_Caspall_Sampled, Max_P_Classifier, Maximum_Breaks, Natural_Breaks, Quantiles, Percentiles, Std_Mean, User_Defined

See PySAL documentation on [Classifier Process](http://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html)

In [10]:
# using default classifier (Natural breaks)

baghdad_hospitals_classifier = ClassifierProcess('num_hospitals', inputs=[baghdad_hospitals])
baghdad_hospitals_classifier.compute()
baghdad_hospitals_classifier_result = baghdad_hospitals_classifier.output.read(format=formats.JSON)

baghdad_hospitals_classifier_result['classifier']

{'classes': {'0 - 1': {'count': 77},
  '1 - 2': {'count': 9},
  '2 - 6': {'count': 2}},
 'classifier_name': 'Natural_Breaks',
 'gadf': 1.0}

In [11]:
# pick colors
current_palette = ["red", "yellow", "green", "blue", "purple", "orange"]
color_dict = {i:current_palette[idx] 
              for idx, i in enumerate(baghdad_hospitals_classifier_result['classifier']['classes'])}
print(color_dict)

# Plot cluster with folium
iraq_map = folium.Map([33.2924177, 44.30871909999998],
                  zoom_start=10.5,
                  tiles='cartodbpositron')

classifier_result = folium.GeoJson(
    baghdad_hospitals_classifier_result,
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['class']],
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    })

iraq_map.add_children(classifier_result)
iraq_map

{'0 - 1': 'red', '1 - 2': 'yellow', '2 - 6': 'green'}


In [12]:
# Pick optimal choropleth classification strategy based on k and GADF (goodness of absolute deviation of fit)

baghdad_hospitals_classifier = ClassifierProcess('num_hospitals', 
                                                 pick_optimal=True,
                                                 inputs=[baghdad_hospitals])
baghdad_hospitals_classifier.compute()
baghdad_hospitals_classifier_result = baghdad_hospitals_classifier.output.read(format=formats.JSON)

baghdad_hospitals_classifier_result['classifier']

{'classes': {'0.0 - 2.0': {'count': 77}, '2.0 - 6.0': {'count': 11}},
 'classifier_name': 'Fisher_Jenks',
 'gadf': 0.89473684210526316}

In [13]:
# pick colors
current_palette = ["red", "yellow", "green", "blue", "purple", "orange"]
color_dict = {i:current_palette[idx] 
              for idx, i in enumerate(baghdad_hospitals_classifier_result['classifier']['classes'])}
print(color_dict)

# Plot cluster with folium
iraq_map = folium.Map([33.2924177, 44.30871909999998],
                  zoom_start=10.5,
                  tiles='cartodbpositron')

classifier_result = folium.GeoJson(
    baghdad_hospitals_classifier_result,
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['class']],
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    })

iraq_map.add_children(classifier_result)
iraq_map

{'0.0 - 2.0': 'red', '2.0 - 6.0': 'yellow'}


In [24]:
# using default classifier (Natural breaks)

baghdad_hospitals_classifier = ClassifierProcess('num_hospitals', 
#                                                  map_classifier = 'Percentiles',
                                                 k=4,
                                                 inputs=[baghdad_hospitals])
baghdad_hospitals_classifier.compute()
baghdad_hospitals_classifier_result = baghdad_hospitals_classifier.output.read(format=formats.JSON)

baghdad_hospitals_classifier_result['classifier']

{'classes': {'0.0 - 2.0': {'count': 77}, '2.0 - 6.0': {'count': 11}},
 'classifier_name': 'Natural_Breaks',
 'gadf': 0.89473684210526316}

In [25]:
baghdad_hospitals_classifier_result

{'classifier': {'classes': {'0.0 - 2.0': {'count': 77},
   '2.0 - 6.0': {'count': 11}},
  'classifier_name': 'Natural_Breaks',
  'gadf': 0.89473684210526316},
 u'features': [{u'geometry': {u'coordinates': [[[[44.25416164813191,
        33.36238209600771],
       [44.25764858728121, 33.3562007038791],
       [44.25634241727511, 33.3558275124488],
       [44.2557826301296, 33.349296662417885],
       [44.258581565857014, 33.35060283242416],
       [44.26138050158448, 33.347244109551184],
       [44.261567097299746, 33.34220602524164],
       [44.263433054451355, 33.34239262095684],
       [44.26697837303942, 33.33343602662899],
       [44.26399284159669, 33.328397942319384],
       [44.26361965016645, 33.32429283658581],
       [44.25634241727511, 33.32391964515557],
       [44.252237311541364, 33.31701560369436],
       [44.24925178009886, 33.31496305082766],
       [44.25205071582633, 33.31235071081528],
       [44.25634241727511, 33.31253730653049],
       [44.26772475590019, 33.30880

# Cluster Process

Calculates the Local Moran's I (Local Indicators of Spatial Association, or LISAs) to identify clusters in data. Returns original vector layer with associated Moran's I statistics

See PySAL documentation on [Local Moran Statistics](http://pysal.readthedocs.io/en/latest/library/esda/moran.html#pysal.esda.moran.Moran_Local)

In [26]:
# Calculate local Moran's I 
baghdad_hospitals_cluster = ClusterProcess('num_hospitals', inputs=[baghdad_hospitals])
baghdad_hospitals_cluster.compute()
baghdad_hospitals_cluster_result = baghdad_hospitals_cluster.output.data

Island id:  [84]


In [27]:
# Plot cluster with folium
iraq_map = folium.Map([33.2924177, 44.30871909999998],
                  zoom_start=11,
                  tiles='cartodbpositron')

# pick colors
current_palette = ["red", "yellow", "green", "blue", "purple", "orange"]
color_dict = {i:current_palette[idx] 
              for idx, i in enumerate(baghdad_hospitals_cluster_result['lm_q'].unique())}
print(color_dict)


{1: 'green', 2: 'yellow', 3: 'red', 4: 'blue'}


In [32]:
import json
json.loads(baghdad_hospitals_cluster_result.to_json())['features'][0]['properties']

{u'DID': 7,
 u'DNAME': u'Mansour',
 u'NID': 74,
 u'NNAME': u'Baghdad International Airport',
 u'class': u'0.0 - 6.0',
 u'index': 0,
 u'lm_Is': 0.0819430257632505,
 u'lm_p_sim': 0.4683,
 u'lm_q': 3,
 u'lm_sig': False,
 u'num_hospitals': 0}

In [28]:

cluster_result = folium.GeoJson(
    baghdad_hospitals_cluster_result,
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['lm_q']],
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    })

iraq_map.add_children(cluster_result)
iraq_map

# Autocorrelation Process

Calculates the Moran's I global autcorrelation for the input data. Uses contiguity weight (queen) by default. Returns Moran's I attributes as json.

See PySAL documentation on [Moran's I Global Autocorrelation Statistic](http://pysal.readthedocs.io/en/latest/library/esda/moran.html#pysal.esda.moran.Moran)

In [29]:
# Calculate Moran's I global autocorrelation
baghdad_hospitals_autocorrelation = AutocorrelationProcess('num_hospitals', inputs=[baghdad_hospitals])
baghdad_hospitals_autocorrelation.compute()
baghdad_hospitals_autocorrelation_result = baghdad_hospitals_autocorrelation.output.data
baghdad_hospitals_autocorrelation_result

Island id:  [84]


{'EI': -0.011363636363636364,
 'EI_sim': -0.011211957397375337,
 'I': 0.0025297567287064578,
 'p_norm': 0.83435951870557812,
 'p_sim': 0.32500000000000001,
 'p_z_sim': 0.38917433971984594,
 'z_sim': 0.28147163702246275}